<a href="https://colab.research.google.com/github/benbaz-2/MPH/blob/main/MPH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# McGill Physics Hackaton 2024: Galaxy shape classification

## Imports

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os
import torch
from torchvision import transforms

## Filtering data

We chose data with a difference in probabilities in class four greater than 0.25

In [14]:
galaxyID=np.loadtxt('/training_solutions_rev1.csv', delimiter = ',', skiprows=1,usecols=0,dtype=int)
#one row containing 61 578 planet IDs

class4=np.loadtxt('/training_solutions_rev1.csv', delimiter = ',', skiprows=1,usecols=(8,9),dtype=float)
data = np.column_stack((galaxyID, class4))
#each subarray corresponds to same galaxy
#remove a row if last two columns are both zero
difference = np.abs(data[:, -1] - data[:, -2])
data=data[difference >= 0.25]


has_spiral=data[:, -2] > data[:, -1]
has_spiral=np.column_stack((data[:,0],has_spiral))
has_spiral=has_spiral.astype(int)
num_spirals=has_spiral[has_spiral[:,-1]==1]


In [15]:
# sampling 1000 data points because we can't fit everything into memory
ids = has_spiral[np.random.choice(has_spiral.shape[0], 1000, replace=False)]

# data will store the numpy arrays in the first column and the labels in the second column
X = np.zeros((1000, 424, 424, 3), int)
y = np.zeros(1000)



In [16]:
# Ennumerate over ids
for i, id in enumerate(ids[:, 0]):
  # Construct the path to retrieve the jpg files
  path = '/content/images/images_training_rev1/' + str(id) + '.jpg'
  with Image.open(path) as img:
    # Append the arrays and labels to X and y arrays
    img_array = np.array(img)
    X[i] = img_array
    y[i] = ids[i, 1]

## Data processing

In [17]:
X = torch.from_numpy(X[1:10]).float()
X = X.permute(0, 3, 1, 2)
X /= 255

In [21]:
means = X.mean(dim=(2, 3), keepdim=True)  # Shape: (10, 3, 1, 1)
stds = X.std(dim=(2, 3), keepdim=True)  # Shape: (10, 3, 1, 1)

# Normalize the tensor
X = (X - means) / (stds + 1e-8)

## Model implementation

1. Create class
2. Decide architecture
3. Train test split
4. Training
5. Hyperparameter tuning
6. Model evaluation
7. Presentation

pytorch tensor datatype. Check pytorch assumption of tensor shape RGB, length width or other way around torchvision